In [1]:
# 라이브러리 임포트
import os
import time
import json
from datetime import datetime, timedelta
import numpy as np
import pandas as pd
#from fbprophet import Prophet
import pickle
import copy
from tqdm.auto import tqdm

In [2]:
from darts.models import *
import darts.utils.timeseries_generation as tg
from darts.utils.likelihood_models import GaussianLikelihood
from darts.datasets import EnergyDataset
from darts.utils.missing_values import fill_missing_values
from darts import TimeSeries
from darts.dataprocessing.transformers import Scaler
from darts.utils.timeseries_generation import datetime_attribute_timeseries
from darts.timeseries import TimeSeries
import warnings

warnings.filterwarnings("ignore")

/ssd/jaewoo/anaconda3/envs/ts/lib/python3.9/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: libtorch_cuda_cu.so: cannot open shared object file: No such file or directory
  warn(f"Failed to load image Python extension: {e}")


In [3]:
%%time
submit = json.load(open('./data/sample_submission/sample_submission.json', 'r', encoding='utf8')) 
df_dic = pickle.load(open('./mydata/df_dic.plk','rb'))
plc_lst = pickle.load(open('./mydata/plc_lst.plk','rb'))
fct_lst = ['pH', 'COD', 'SS', 'N', 'P', 'T']

CPU times: user 750 ms, sys: 156 ms, total: 906 ms
Wall time: 906 ms


In [4]:
# Prophet 예측을 위한 dataframe 포맷 제작
future = pd.DataFrame([str(x)[:10] for x in list(pd.date_range(start='2018-02-01', end='2019-12-31', inclusive="both"))], columns=['ds'])
future['y'] = np.nan
#future

In [5]:
from darts.models.forecasting import tcn_model

In [ ]:
# 추론 : tcn_model
start_time = time.time()

for i in tqdm(plc_lst):
    plc_df = df_dic[i]
    
    for j in fct_lst: # 피쳐 리스트
        fct_df = plc_df[['ds', j]]   # 날짜 - 피쳐1 조합임.
        
        fct_df = fct_df.rename(columns={j:'y'}) # 피쳐1을 y로 바꾼다.
        fct_df['ds'] = fct_df.ds.apply(lambda x : datetime.strptime(x , '%Y%m%d'))
        fct_df = fct_df.dropna()
        fct_df['y'] = pd.to_numeric(fct_df.y)
        
        df_align = pd.DataFrame([ x for x in list(pd.date_range( end = '2018-01-31', periods = len(fct_df), inclusive="both"))], columns=['ds'])
        df_align['y'] = fct_df.y.values
        
        # TimeSeris 로 변환 하기 
        dfts = TimeSeries.from_dataframe(df_align,  time_col = 'ds' , value_cols  = ['y'], fill_missing_dates=True  , freq='D')
        
        diff = datetime(2018,2,1) - df_align.loc[0].ds
        if diff.days > 720:
            lag = 700
        else:
            lag = diff.days - 10
            
        my_model = tcn_model.TCNModel(
            input_chunk_length = 60,
            output_chunk_length = 1,
            n_epochs=12,
            kernel_size =3,
            #num_filters = 12,
            dropout = 0.3,
            batch_size = 64,
            #pl_trainer_kwargs  = {"accelerator": "gpu", "gpus": -1, "auto_select_gpus": True} ,
            dilation_base=2,
            weight_norm=True,
            random_state=2022,
            
            
        )
        
        my_model.fit(dfts)
        pred_series = my_model.predict( n = 699 )
        
        forecast = pred_series.pd_dataframe()
        forecast['ds'] = forecast.index
        forecast['ds'] = forecast['ds'].apply(lambda x : str(x).split(' ')[0].split('-')[0]+str(x).split(' ')[0].split('-')[1]+str(x).split(' ')[0].split('-')[2])
        forecast.columns = ['yhat', 'ds']
        forecast = forecast.reset_index(drop=True)
        
        for idx,k in enumerate(submit[i].keys()):
            submit[i][k][j] = np.round(forecast.yhat[idx],6)
      
        
end_time = time.time()
print('소요시간 :', (end_time - start_time)/60, 'minutes')

---
## 

In [5]:
from darts.models.forecasting import random_forest

In [ ]:
# 추론 : rf
start_time = time.time()

for i in tqdm(plc_lst):
    plc_df = df_dic[i]
    
    for j in fct_lst: # 피쳐 리스트
        fct_df = plc_df[['ds', j]]   # 날짜 - 피쳐1 조합임.
        
        fct_df = fct_df.rename(columns={j:'y'}) # 피쳐1을 y로 바꾼다.
        fct_df['ds'] = fct_df.ds.apply(lambda x : datetime.strptime(x , '%Y%m%d'))
        fct_df = fct_df.dropna()
        fct_df['y'] = pd.to_numeric(fct_df.y)
        
        df_align = pd.DataFrame([ x for x in list(pd.date_range( end = '2018-01-31', periods = len(fct_df), inclusive="both"))], columns=['ds'])
        df_align['y'] = fct_df.y.values
        
        # TimeSeris 로 변환 하기 
        dfts = TimeSeries.from_dataframe(df_align,  time_col = 'ds' , value_cols  = ['y'], fill_missing_dates=True  , freq='D')
        
        diff = datetime(2018,2,1) - df_align.loc[0].ds
        if diff.days > 720:
            lag = 700
        else:
            lag = diff.days - 10
            
        my_model = random_forest.RandomForest(
            lags=lag,
            output_chunk_length=1,
            n_estimators = 100,
            random_state= 2022,
            n_jobs = 42
        )
        
        my_model.fit(dfts)
        pred_series = my_model.predict( n = 699 )
        
        forecast = pred_series.pd_dataframe()
        forecast['ds'] = forecast.index
        forecast['ds'] = forecast['ds'].apply(lambda x : str(x).split(' ')[0].split('-')[0]+str(x).split(' ')[0].split('-')[1]+str(x).split(' ')[0].split('-')[2])
        forecast.columns = ['yhat', 'ds']
        forecast = forecast.reset_index(drop=True)
        
        for idx,k in enumerate(submit[i].keys()):
            submit[i][k][j] = np.round(forecast.yhat[idx],6)
      
        
end_time = time.time()
print('소요시간 :', (end_time - start_time)/60, 'minutes')

  0%|          | 0/545 [00:00<?, ?it/s]